In [32]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime,timedelta,date
from collections import defaultdict

In [129]:
df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time','host','consumer_group'],parse_dates=['time'])
#df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time'],parse_dates=['time'])

In [122]:
#host_cons=[(host,group) for host,group in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values]


In [ ]:
# добавить признаки свертки

In [39]:
#ts_period=1 # количество дней 
#n_length=24 # количество часов


#def window_stack(a, window=ts_period*n_length):
    # разбиваем данные на окна длиной ts_period
#    X=np.array([a[i:window+i] for i in range(int(len(a) - window))])
#    return X



In [165]:
outliers_fraction = 0.005
def predict(model,X,mode=0):
    if mode==0:
        return model.fit(X).predict(X)
    else:
        return model.fit_predict(X)
        


In [162]:
from sklearn.ensemble import IsolationForest

rns = np.random.RandomState(42)
model_IF = IsolationForest(random_state=rns,contamination=outliers_fraction,n_estimators =1000)

anomaly_IF = predict(model_IF,df)



In [170]:
from sklearn.neighbors import LocalOutlierFactor

model_LOF = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)
anomaly_LOF = predict(model_LOF,df,mode=1)

In [172]:
from sklearn import svm

model_SVM = svm.OneClassSVM(random_state=rns, nu=outliers_fraction, kernel="rbf", gamma=0.01)
anomaly_SVM = predict(model_SVM,df)

In [173]:
from sklearn.covariance import EllipticEnvelope

model_EE = EllipticEnvelope(contamination=outliers_fraction)
anomaly_EE = predict(model_EE,df)

C:\Users\JustMe\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [193]:
from sklearn.decomposition import PCA

model_PCA = PCA(n_components=15, svd_solver='full')
model_PCA.fit(df)

PCA(copy=True, iterated_power='auto', n_components=15, random_state=None,
  svd_solver='full', tol=0.0, whiten=False)

In [212]:
w_components_ = np.ones([model_PCA.n_components_, ])
selected_components_ = model_PCA.components_[-1 * model_PCA.n_selected_components_:, :]
selected_w_components_ = w_components_[-1 * model_PCA.n_selected_components_:]

AttributeError: 'PCA' object has no attribute 'n_selected_components_'

In [186]:
np.unique(anomaly_GMM,return_counts =True)

(array([0, 1], dtype=int64), array([10992,  7680], dtype=int64))

In [178]:
df_predict=pd.concat([df.reset_index(),pd.DataFrame(anomaly_GMM,columns=['is_anomaly'])],axis=1,sort=False)
df_predict[(df_predict.is_anomaly==-1)&(df_predict.consumer_group=='szb_sandbox_group')]

,time,host,consumer_group,cpu,dbtime,iowait,oth_wait,pga,pxenq,pxqry,...,hour_of_day_23,hour_of_day_3,hour_of_day_4,hour_of_day_5,hour_of_day_6,hour_of_day_7,hour_of_day_8,hour_of_day_9,quarter_of_year_4,is_anomaly


In [ ]:
df_predict=pd.concat([df.reset_index(),pd.DataFrame(anomaly_IF,columns=['is_anomaly'])],axis=1,sort=False)
df_predict[(df_predict.is_anomaly==-1)&(df_predict.consumer_group=='szb_sandbox_group')]